## Prerequisites

In [1]:
!pip install numpy matplotlib pandas sklearn

In [2]:
!python -V

Python 3.8.8


## Data Preprocessing

In [3]:
from scipy.io import loadmat

In [4]:
import pickle

In [5]:
import h5py

In [6]:
data = loadmat(r'ST640-GaN-Si-50-generate TTR not shift.mat')

In [7]:
import scipy

In [8]:
X = data['re']

In [9]:
X.shape

(125000, 300)

In [10]:
import pickle
import numpy as np

In [11]:
np.set_printoptions(suppress=True)

In [12]:
X = np.transpose(data['re'])

In [13]:
X.shape

(300, 125000)

In [14]:
X = X.T

In [15]:
X.shape

(125000, 300)

In [16]:
X = X[:,20:299]

In [17]:
X.shape

(125000, 279)

In [18]:
layer_2 = np.linspace(10, 250, 50, endpoint=True)
layer_3 = np.linspace(20, 200, 50, endpoint=True)
layer_4 = np.linspace(50, 250, 50, endpoint=True)

In [19]:
y = np.zeros((X.shape[0], 3))

In [20]:
i = 0
for l4 in layer_4:
    for l3 in layer_3:
        for l2 in layer_2:
            y[i] = [l4, l3, l2]
            i = i + 1

In [21]:
y.shape

(125000, 3)

### Train test split

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [24]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

In [25]:
#property = 0

In [26]:
x_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

x_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [27]:
# Builds dataset with ALL data
origin_train_dataset = TensorDataset(x_train_tensor, y_train_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(origin_train_dataset, [int(x_train_tensor.shape[0] * 0.9), int(x_train_tensor.shape[0] * 0.1)])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=2000)
val_loader = DataLoader(dataset=val_dataset, batch_size=2000)

test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader  = DataLoader(dataset=test_dataset, batch_size=2000)

## Model

In [28]:
import torch.nn as nn

In [29]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm1d(X.shape[1])
        self.fc1 = nn.Linear(X.shape[1], 100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.bn2(x)
        x = self.fc2(x)
        x = torch.tanh(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        return x

## Training

In [30]:
import torch.optim as optim

In [31]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [32]:
n_epochs = 80

In [33]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yh = model(x)
        #yh = torch.reshape(yh, (-1,))
        loss = loss_fn(y, yh)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

In [34]:
model = Net().to(device)

loss_fn = nn.MSELoss(reduction='mean')

# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

train_step = make_train_step(model, loss_fn, optimizer)

In [35]:
model.eval()

Net(
  (bn1): BatchNorm1d(279, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=279, out_features=100, bias=True)
  (bn2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=3, bias=True)
)

In [36]:
training_losses = []
validation_losses = []

for epoch in range(n_epochs):
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yh = model(x_val)
            #yh = torch.reshape(yh, (-1,))
            val_loss = loss_fn(y_val, yh).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.5f}\t Validation loss: {validation_loss:.5f}")

[1] Training loss: 20704.31492	 Validation loss: 20452.68132
[2] Training loss: 20174.04832	 Validation loss: 19754.61589
[3] Training loss: 19358.79232	 Validation loss: 18817.06348
[4] Training loss: 18294.46434	 Validation loss: 17663.85905
[5] Training loss: 17061.75613	 Validation loss: 16408.31217
[6] Training loss: 15684.45446	 Validation loss: 15138.37874
[7] Training loss: 13906.45115	 Validation loss: 13074.32031
[8] Training loss: 12364.54489	 Validation loss: 11594.61377
[9] Training loss: 10934.84194	 Validation loss: 10239.41536
[10] Training loss: 9657.79554	 Validation loss: 9067.29964
[11] Training loss: 8592.00551	 Validation loss: 8138.81470
[12] Training loss: 7777.05649	 Validation loss: 7457.13493
[13] Training loss: 7220.30688	 Validation loss: 7035.76302
[14] Training loss: 6861.19438	 Validation loss: 6751.49398
[15] Training loss: 6656.59467	 Validation loss: 6615.46615
[16] Training loss: 6536.45888	 Validation loss: 6526.31071
[17] Training loss: 6438.69590	

In [37]:
# model.state_dict()

## Testing

In [38]:
def mean_absolute_percentage_error(y_true, y_pred):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100

In [39]:
x_test_tensor = x_test_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)
y_pred = model(x_test_tensor).squeeze()

In [40]:
test_loss = loss_fn(y_test_tensor, y_pred)
print(test_loss)

tensor(24.1154, grad_fn=<MseLossBackward0>)


In [41]:
print(f"The mean of absolute percentage error: {mean_absolute_percentage_error(y_test_tensor.cpu(), y_pred.cpu()):.2f}%")

The mean of absolute percentage error: 4.32%


## Real-world case

In [42]:
import pandas as pd

In [43]:
case1 = pd.read_csv('20230220036.csv', header=None)

In [44]:
case1.shape

(1, 300)

In [45]:
case1= np.array(case1)
case1=case1[:,20:299]
case1_tensor = torch.from_numpy(case1).float()

In [46]:
model.eval()
pred_1 = model(case1_tensor)

In [47]:
pred_1

tensor([[165.0116,  69.0174, 126.4408]], grad_fn=<ReluBackward0>)